## POSTGRESQL 설치 방법
처음 postgresql을 사용하는 경우, Psycopg2와 postgres를 설치하여 super user 선언 및 사용자 권한 부여를 진행해야함. 다만, 주피터 노트북에서 사용하는 경우 설치 및 실행 과정이 다름.

- **Psycopg2-binary 설치**(PostgreSQL과 연결하는 Python Package)
> ```bash
> pip install psycopg2-binary
> conda install -c conda-forge postgresql
> psql --version
> ```

- **PSQL 설치** (PostgreSQL 그 자체)
> ```bash
> conda install -c conda-forge postgresql
>```
- 설치 완료 후 버전 확인
>``` bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> conda list | findstr postgresql
> postgresql                16.3                 h7f155c9_0    conda-forge
> ```
 
- **pgdata** 받아오기
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\initdb" -D "./pgdata"
> ...(log 중략)...
> Success. You can now start the database server using:
>
>   "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l logfile start
> ```
실행 이후 해당 프로젝트 내부에 *pgdata* 디렉토리가 만들어졌는지 확인할 것.

## POSTGRESQL 사용자 선언
SQL을 사용하기 위헤서는 Super User와 하부 사용자가 필요. 여기서 Super User의 이름은 *postgres*로, 하부 사용자 이름은 *d519*로 설정. 먼저, 암호 코드 설정
**1. 암호 파일 생성**
> ```bash
> echo "d519" > superuser_password.txt
> ```

**2. 오류 방지를 위해 기존의 **pgdata** 삭제 (삭제 안될 경우 파일 탐색기에서 pgdata파일 삭제)**
>```bash
> rmdir /s pgdata
> ```

**3. 새로 초기화**
> ```bash
> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> ```
- 실행 시 로그
> > ```bash
> > (branchPJH) PS C:\junha\Tilde_Chatbot> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> > The files belonging to this database system will be owned by user "overr".
> > This user must also own the server process.
> > 
> > The database cluster will be initialized with locale "Korean_Korea.949".
> > Encoding "UHC" implied by locale is not allowed as a server-side encoding.
> > The default database encoding will be set to "UTF8" instead.
> > initdb: could not find suitable text search configuration for locale "Korean_Korea.949"
> > The default text search configuration will be set to "simple".
> > 
> > Data page checksums are disabled.
> > 
> > Enter new superuser password: 
> > Enter it again:
> > 
> > creating directory pgdata ... ok
> > creating subdirectories ... ok
> > selecting dynamic shared memory implementation ... windows
> > selecting default max_connections ... 100
> > selecting default shared_buffers ... 128MB
> > selecting default time zone ... Asia/Seoul
> > creating configuration files ... ok
> > running bootstrap script ... ok
> > performing post-bootstrap initialization ... ok
> > syncing data to disk ... ok
> > 
> > Success. You can now start the database server using:
> > 
> >     ^"C^:^\Users^\overr^\anaconda3^\envs^\branchPJH^\Library^\bin^\pg^_ctl^" -D ./pgdata -l logfile start
> > ```
   
**4. `initdb` 명령 실행**
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
- 실행 시 로그
> > ```bash
> > waiting for server to start.... done
> > server started
> > ```

# PostgreSQL 내부 작업

**1. Postgres 실행 확인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> netstat -an | Select-String "5432"
> TCP    127.0.0.1:5432         0.0.0.0:0              LISTENING
> TCP    [::1]:5432             [::]:0                 LISTENING
> ```

**2. 로그인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> psql -U postgres -h localhost -p 5432
> Password for user postgres: 
> psql (16.3)
> Type "help" for help.
> postgres=#
> ```
`postgres=#`가 나오면 로그인 성공.

**3. User 추가**
> ```bash
> postgres=# CREATE ROLE d519 WITH LOGIN PASSWORD 'd519';
> CREATE ROLE
> ```

**4. DB 선언**
> ```bash
> postgres=# CREATE DATABASE TildeChatBotDB; 
> CREATE DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
> (4 rows)
> ```

**5. tildechatbotdb 소유주를 postgres에서 d519로 변경**
> ```bash
> postgres=# ALTER DATABASE tildechatbotdb OWNER TO d519;
> ALTER DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | d519     | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =Tc/d519             +
>                 |          |          |                 |                  |                  |            |           | d519=CTc/d519
> (4 rows)
> ```

**6. 로그아웃**
> ```bash
> postgres=# \q
> (branchPJH) PS C:\junha\Tilde_Chatbot>
> ```


In [65]:
import pandas as pd
import psycopg2

# 전원 다시 시작시
- Terminal에서 PostgerSQL 서버 시작하기  
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
 
- Terminal에서 super user인 **postgres**로 로그인
> ```bash
> psql -U postgres -h localhost -p 5432
> ```

In [66]:
# & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start

In [67]:
# Postgres 연결 확인
DB_CONFIG = {
        'host': 'localhost',
        'database': 'tildechatbotdb',
        'user': 'd519',
        'password': 'd519',
        'port': 5432
    }

try:
    # PostgreSQL에 연결
    conn = psycopg2.connect(**DB_CONFIG)
    print("PostgreSQL 연결 성공!")
    conn.close()
except Exception as e:
    print(f"PostgreSQL 연결 실패: {e}")


PostgreSQL 연결 성공!


## 초기 작업 : 엑셀 내용 삽입
2024년 12월 29일 기준으로 PostgreSQL 내부 tildechatbotdb에는 ChatTable이라는 이름의 테이블이 존재. 엑셀 내용이 전부 삽입되어있는 상태임. 만약, 새로 선언한다면, 주석을 제거하고 실행할 것.

In [68]:
# def create_table_from_excel_psycopg2(excel_file, table_name, db_config):
#     df = pd.read_excel(excel_file)
# 
#     conn = psycopg2.connect(**db_config)
#     cursor = conn.cursor()
# 
#     columns = ', '.join([f'"{col}" TEXT' for col in df.columns])
#     create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})'
#     cursor.execute(create_table_query)
# 
#     for _, row in df.iterrows():
#         column_names = ', '.join([f'"{col}"' for col in df.columns])
#         placeholders = ', '.join(['%s'] * len(row))
#         insert_query = f'INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})'
#         cursor.execute(insert_query, tuple(row.values))
# 
#     conn.commit()
#     cursor.close()
#     conn.close()
#     print(f"Table '{table_name}' created and data inserted successfully.")

In [69]:
# EXCEL_FILE = "C:/junha/Datasets/ChatData.xlsx"
# TABLE_NAME = "ChatTable"
# DB_CONFIG = {
#         'host': 'localhost',
#         'database': 'tildechatbotdb',
#         'user': 'd519',
#         'password': 'd519',
#         'port': 5432
#     }
# 
# create_table_from_excel_psycopg2(EXCEL_FILE, TABLE_NAME, DB_CONFIG)


## 데이터베이스 삽입 상태 확인

In [78]:
def verify_data_insertion(table_name, db_config):
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)

        result = cursor.fetchall()

        if not result:
            print(f"No data found in table '{table_name}'.")
            return None

        column_names = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(result, columns=column_names)
        print(f"Data in table '{table_name}':")

        cursor.close()
        conn.close()
        return df

    except Exception as e:
        print(f"Error verifying data: {e}")


In [79]:
verify_data_insertion('ChatTable',DB_CONFIG)

Error verifying data: column "유독가스" does not exist
LINE 1: SELECT * FROM ChatTable where 노출제품명 like 유독가스
                                                      ^


## 원하는 값만 추출

In [72]:
def select_distinct_values(query, db_config):
    try:
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        
        distinct_values = [row[0] for row in results]
        return distinct_values
    except Exception as e:
        print("Database error:", e)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [80]:
TBL_NAME = "ChatTable"
COLUMN_NAME = "노출제품명"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
distinct_values = select_distinct_values(QUERY, DB_CONFIG)

matching_rows = select_distinct_values(distinct_values, TBL_NAME, COLUMN_NAME, DB_CONFIG)
matching_rows

TypeError: select_distinct_values() takes 2 positional arguments but 4 were given

In [73]:
TBL_NAME = "ChatTable"
COLUMN_NAME = "노출제품명"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
distinct_values = select_distinct_values(QUERY, DB_CONFIG)
distinct_values

['유독가스',
 '자동차 워셔액',
 '살충스프레이',
 '보온팩(발열체)',
 '플라스틱 공장 연기',
 '공업용 엔진 세척제',
 '올편제에스',
 '돈나무',
 '자동차용 방향제',
 '에폭시 접착제',
 '아론',
 '인화성 화학물질',
 'escitalopram',
 '방향제 펌핑식 스프레이',
 '연료 첨가제',
 '수면제',
 '클로자핀',
 '벽 코팅용 도배 풀',
 '금속성 페인트',
 '판베시',
 '문구용 잉크 스탬프 패드',
 '우울증약(벤조디아제핀계열 추정)',
 '자동차 대시보드 클리너 스프레이',
 '렉사프로정',
 '심장 강화 영양제',
 '주방용 화학 세제',
 '다이어트 약, 에너지 드링크',
 '공사장에서 날아오는 연기',
 '자동차 페인트',
 '방향제 캡슐',
 '욕실 청소용 스프레이',
 '명인탄산리튬정',
 'lithium',
 '욕실 청소용 분말 세정제',
 '클로나제팜',
 '스리반정(lorazepam), 아고틴정(agomelatine)',
 '섬유유연제',
 '로멜라인',
 '스틸녹스 10mg',
 '육류 보관용 플라스틱 포장재',
 '보령부스파정',
 '졸민정0.25mg, 자낙스정0.5mg, 트리티코정50mg, 쎄로켈정100mg, 프리스틱서방정 100mg, 프리스틱서방정 50mg',
 '불안증 약',
 '자디팜',
 '디퓨저 액체',
 '도료 폐기물, 락스 냄새가 나는 화학약품',
 '금속 세정제',
 '보습 크림',
 'puri bag (물 정화 키트)',
 '네일 리무버',
 '우드스테인(목재용 코팅제)',
 '엔진룸 청소용 세정제',
 '탈취제 스프레이',
 '염분 보충제',
 '에릭손정',
 '제습제',
 '리스카펜정, 메가슬림정, 미세틴 캡슐, 하이라제정, 에닝정, 리치칼비정, 디아제팜정',
 '진통제, 감기약',
 '고무 연기',
 'Norfloxacin',
 '진통제, 염증 억제제, 마늘 추출물 건강기능식품',
 '멕시부펜 해열제',
 '자동차용품 세정제',
 '트라마돌',
 '초콜릿, 콜라, 과자

In [91]:
question = "우리 아이가 포름알데히드 먹었어요. 어떡해요!"

if any(value in question for value in distinct_values):
    print(distinct_values) 
else:
    print("nope")


nope
